In [1]:
from nustar_lunar_pointing import planning
import astropy.units as u

In [2]:
from pyorbital import tlefile

In [4]:
tleloc = '../data/NUSTAR_TLE_ARCHIVE.txt.2016122'
tle = tlefile.read('NuSTAR', tleloc)

KeyError: "Found no TLE entry for 'NUSTAR'"

# Set up the offset you want to use here:

The first element is the directory +West of the center of the Sun, the second is the offset +NORTH of the center of the Sun. If you want multiple pointing locations you can either specify an array of offsets or do this "by hand" below.

In [ ]:
offset = [-190., -47.]*u.arcsec

In [ ]:

for ind, orbit in enumerate(orbits):
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_sky_position(midTime, offset)
    print("Orbit: {}".format(ind))
    print("Orbit start: {} Orbit end: {}".format(orbit[0].isoformat(), orbit[1].isoformat()))
    print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
    print("")

# Sanity check: Reproduce the 2016-07-26 pointing.

Based on the IDL version of this code, when the aim time was 2016-07-26 19:53:15.00 the NP was 8.86 degrees and the RA and Dec were 126.04630 and 19.33532 deg, respectively for an offset of +1000 arcseconds West and +150 arcseconds North of the center of the Sun.

In [ ]:
aim_time = '2016-07-26T19:53:15.00'
offset = [1000, 150]*u.arcsec
sky_pos = planning.get_sky_position(aim_time, offset)
print(sky_pos)

In [ ]:
np = planning.get_nustar_roll(aim_time, 0)
print(np)